In [1]:
# 데이터 처리 라이브러리
import os
import os.path as pth
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split

# Tensorflow 관련 라이브러리
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Dropout, BatchNormalization, PReLU
from tensorflow.keras.layers import Flatten, Activation, Dense, GlobalAveragePooling2D, Softmax
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

# tensorflow에서 사용할 수 없는 기능을 구현하는 기여 저장소
from tensorflow_addons import optimizers


# GPU 설정
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    # 특정 GPU에 1GB 메모리만 할당하도록 제한
    try:
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')
        
        tf.config.experimental.set_memory_growth(gpus[0], True)
        
#         tf.config.experimental.set_virtual_device_configuration(
#             gpus[0],
#             [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=15000)])
    except RuntimeError as e:
    # 프로그램 시작시에 가상 장치가 설정되어야만 합니다
        print(e)

In [2]:
train_tfrecord_path = pth.join('./data/emotion_crop_tf', 'tf_record_train.tfrecords')
valid_tfrecord_path = pth.join('./data/emotion_crop_tf', 'tf_record_valid.tfrecords')

# BUFFER_SIZE, BATCH_SIZE
BUFFER_SIZE = 256
BATCH_SIZE = 50
NUM_CLASS = 8



image_feature_description = {
    'image_raw': tf.io.FixedLenFeature([], tf.string),
    'img_id': tf.io.FixedLenFeature([], tf.string),
}



def _parse_image_function(example_proto):
    return tf.io.parse_single_example(example_proto, image_feature_description)

def map_func(target_record):
    img = target_record['image_raw']
    label = int(target_record['img_id'])
    img = tf.image.decode_jpeg(img, channels=3)
    img = tf.dtypes.cast(img, tf.float32)
    return img, label

def prep_func(image, label):
    print(type(label))

    result_image = image / 255
    result_image = tf.image.resize(result_image, (224,224))

    onehot_label = tf.one_hot(label, depth=NUM_CLASS)
    return result_image, onehot_label

    

dataset = tf.data.TFRecordDataset(train_tfrecord_path, compression_type='GZIP')
dataset = dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
dataset = dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

valid_dataset = tf.data.TFRecordDataset(valid_tfrecord_path, compression_type='GZIP')
valid_dataset = valid_dataset.map(_parse_image_function, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.map(map_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.shuffle(BUFFER_SIZE)
valid_dataset = valid_dataset.batch(BATCH_SIZE)
valid_dataset = valid_dataset.map(prep_func, num_parallel_calls=tf.data.experimental.AUTOTUNE)
valid_dataset = valid_dataset.prefetch(buffer_size=tf.data.experimental.AUTOTUNE)

<class 'tensorflow.python.framework.ops.Tensor'>
<class 'tensorflow.python.framework.ops.Tensor'>


In [3]:
earlystop = EarlyStopping(patience = 5)

learning_rate_reduction = ReduceLROnPlateau(monitor = 'val_accuracy',
                                            patience = 2,
                                            factor = 0.5,
                                            min_lr = 1e-7,
                                            verbose = 1)

model_check = ModelCheckpoint(filepath = './model/facial_mobile_224_224_PReLU.h5',
                              monitor = 'val_loss',
                              save_best_only = True)

callbacks = [earlystop, learning_rate_reduction, model_check]

In [5]:
from tensorflow.keras.applications import MobileNetV2
mobilenet = MobileNetV2(weights = 'imagenet', include_top = False, input_shape = (224,224,3))


for layer in mobilenet.layers:
    layer.trainable = True
    
for (i, layer) in enumerate(mobilenet.layers):
    print(str(i), layer.__class__.__name__, layer.trainable)
    
    
model = Sequential()
model.add(mobilenet)
model.add(GlobalAveragePooling2D())
model.add(Dense(8))
model.add(BatchNormalization())
model.add(PReLU())
model.add(Softmax(dtype='float32', name='softmax'))
model.summary()

0 InputLayer True
1 ZeroPadding2D True
2 Conv2D True
3 BatchNormalization True
4 ReLU True
5 DepthwiseConv2D True
6 BatchNormalization True
7 ReLU True
8 Conv2D True
9 BatchNormalization True
10 Conv2D True
11 BatchNormalization True
12 ReLU True
13 ZeroPadding2D True
14 DepthwiseConv2D True
15 BatchNormalization True
16 ReLU True
17 Conv2D True
18 BatchNormalization True
19 Conv2D True
20 BatchNormalization True
21 ReLU True
22 DepthwiseConv2D True
23 BatchNormalization True
24 ReLU True
25 Conv2D True
26 BatchNormalization True
27 Add True
28 Conv2D True
29 BatchNormalization True
30 ReLU True
31 ZeroPadding2D True
32 DepthwiseConv2D True
33 BatchNormalization True
34 ReLU True
35 Conv2D True
36 BatchNormalization True
37 Conv2D True
38 BatchNormalization True
39 ReLU True
40 DepthwiseConv2D True
41 BatchNormalization True
42 ReLU True
43 Conv2D True
44 BatchNormalization True
45 Add True
46 Conv2D True
47 BatchNormalization True
48 ReLU True
49 DepthwiseConv2D True
50 BatchNormaliza

In [6]:
model.compile(loss='categorical_crossentropy',
              optimizer=optimizers.RectifiedAdam(learning_rate=0.001),
              metrics=['accuracy'])

In [7]:
history = model.fit(dataset,
                    epochs=1000,
                    validation_data=valid_dataset,
                    callbacks=callbacks,
                   )

Epoch 1/1000
1503/1503 [==============================] - 609s 405ms/step - loss: 0.5238 - accuracy: 0.7788 - val_loss: 1.1308 - val_accuracy: 0.5495 - lr: 0.0010
Epoch 2/1000
1503/1503 [==============================] - 611s 407ms/step - loss: 0.2951 - accuracy: 0.8168 - val_loss: 1.7646 - val_accuracy: 0.2982 - lr: 0.0010
Epoch 3/1000
1503/1503 [==============================] - ETA: 0s - loss: 0.2715 - accuracy: 0.8074
Epoch 00003: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
1503/1503 [==============================] - 608s 405ms/step - loss: 0.2715 - accuracy: 0.8074 - val_loss: 3.5765 - val_accuracy: 0.1906 - lr: 0.0010
Epoch 4/1000
1503/1503 [==============================] - 611s 406ms/step - loss: 0.1304 - accuracy: 0.8457 - val_loss: 0.6831 - val_accuracy: 0.7302 - lr: 5.0000e-04
Epoch 5/1000
1503/1503 [==============================] - 608s 405ms/step - loss: 0.0960 - accuracy: 0.8514 - val_loss: 1.4653 - val_accuracy: 0.5576 - lr: 5.0000e-04
Epoch 6/10